# Complexity scaling of calibration

In [1]:
%load_ext autoreload

In [2]:
%autoreload 1

In [13]:
import os
import sys
import time
import random
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
from scipy.special import softmax
from sklearn.isotonic import IsotonicRegression
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Activation, Input
from tensorflow.keras.optimizers import Adam, SGD

%aimport utils
%aimport utils.data
%aimport utils.ops
%aimport utils.metrics
%aimport utils.visualization
%aimport flows.nice
%aimport calibrators
from utils.data import get_cifar10, get_cifar3, load_logits
from utils.ops import onehot_encode, optim_temperature, detection_log_likelihood_ratios
from utils.metrics import neg_log_likelihood, expected_calibration_error
from utils.visualization import plot_pdf_simplex, plot_prob_simplex, reliability_plot
from utils.visualization import reliability_diagram, ECE_plot, plot_cal_regions_ternary
from flows.nice import AddCouplingLayer, MLP, ReIndex
from calibrators import PAVCalibrator, MLRCalibrator, NiceCalibrator, TempScalingCalibrator

In [14]:
%matplotlib inline
np.random.seed(1)

In [15]:
cifar_dir = '../cifar-10'
models_dir = '../pretrained-models'

## CNN trained on CIFAR3 vs CNN trained on CIFAR10

### Loading datasets

In [16]:
cifar3, ix2label3 = get_cifar3(cifar_dir, test=True)
target3 = onehot_encode(cifar3['test_labels'])

cifar10, ix2label10 = get_cifar10(cifar_dir, test=True)
target10 = onehot_encode(cifar10['test_labels'])

### Loading models

In [17]:
model_c3 = 'cnn_cifar3'
model_c10 = 'resnet32_v1'

_, logits3 = load_logits(os.path.join(models_dir, model_c3))
probs3 = softmax(logits3, axis=1)

_, logits10 = load_logits(os.path.join(models_dir, model_c10))
probs10 = softmax(logits10, axis=1)